Simple Chatbot Using Langgraph

In [2]:
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END

from typing import Annotated
from langgraph.graph.message import add_messages

State

In [4]:
class State(TypedDict):
    messages:Annotated[list, add_messages]

Import API Keys

In [9]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")


In [12]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model = "gpt-4o", temperature = 0)

In [20]:
from langchain_groq import ChatGroq

llm_groq = ChatGroq (model = "llama-3.1-8b-instant")

Nodes

In [21]:
def superbot(state:State):
    return {"messages": [llm_groq.invoke(state['messages'])]}

In [22]:
graph = StateGraph(State)

graph.add_node("superbot", superbot)

graph.add_edge(START, "superbot")
graph.add_edge("superbot", END) 

graph_builder = graph.compile()

Graph Invocation

In [23]:
graph_builder.invoke({"messages":"Hi, My name is Kunal"})

{'messages': [HumanMessage(content='Hi, My name is Kunal', additional_kwargs={}, response_metadata={}, id='d97994a5-a7f3-4dac-a152-0d3d402d2c6e'),
  AIMessage(content="Nice to meet you, Kunal. I'm an AI assistant, here to help you with any questions or information you might need. What's on your mind today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 42, 'total_tokens': 77, 'completion_time': 0.062280606, 'prompt_time': 0.002396642, 'queue_time': 0.187573696, 'total_time': 0.064677248}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_c54ae48ece', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--c88780df-3427-493e-8ef2-957a29231cdd-0', usage_metadata={'input_tokens': 42, 'output_tokens': 35, 'total_tokens': 77})]}

Streaming the responses

In [26]:
for event in graph_builder.stream({"messages":"Hi, My name is Kunal "}):
    print(event)

{'superbot': {'messages': [AIMessage(content='Nice to meet you, Kunal. Is there something I can help you with or would you like to chat?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 43, 'total_tokens': 67, 'completion_time': 0.025533161, 'prompt_time': 0.002693709, 'queue_time': 0.187381407, 'total_time': 0.02822687}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_8804b970d6', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--0b9d2a2a-45a6-4fa5-a1e9-f27f674d6843-0', usage_metadata={'input_tokens': 43, 'output_tokens': 24, 'total_tokens': 67})]}}
